In [75]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

In [ ]:
data = pd.read_csv('USA_Final3.csv')

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41290 entries, 0 to 41289
Data columns (total 9 columns):
title          41290 non-null object
year           41290 non-null int64
director       41290 non-null object
actor1         41290 non-null object
actor2         41290 non-null object
genre          41290 non-null object
rating         41290 non-null float64
country        41290 non-null object
releasedate    36521 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 2.8+ MB


In [78]:
train_data = data[0:35000]
test_data = data[35000:]

In [79]:
del train_data['releasedate']
del test_data['releasedate']


In [81]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 8 columns):
title       35000 non-null object
year        35000 non-null int64
director    35000 non-null object
actor1      35000 non-null object
actor2      35000 non-null object
genre       35000 non-null object
rating      35000 non-null float64
country     35000 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6290 entries, 35000 to 41289
Data columns (total 8 columns):
title       6290 non-null object
year        6290 non-null int64
director    6290 non-null object
actor1      6290 non-null object
actor2      6290 non-null object
genre       6290 non-null object
rating      6290 non-null float64
country     6290 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 393.2+ KB


In [82]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
       # Encoding only categorical variables
#if test_data.column=='genre':
       # Using whole data to form an exhaustive list of levels
data2=train_data['genre'].append(test_data['genre'])
le.fit((data2))
train_data['genre']=le.transform(train_data['genre'])
test_data['genre']=le.transform(test_data['genre'])


C:\Users\vijay\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\vijay\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [83]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 8 columns):
title       35000 non-null object
year        35000 non-null int64
director    35000 non-null object
actor1      35000 non-null object
actor2      35000 non-null object
genre       35000 non-null int64
rating      35000 non-null float64
country     35000 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6290 entries, 35000 to 41289
Data columns (total 8 columns):
title       6290 non-null object
year        6290 non-null int64
director    6290 non-null object
actor1      6290 non-null object
actor2      6290 non-null object
genre       6290 non-null int64
rating      6290 non-null float64
country     6290 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 393.2+ KB


In [84]:
features = train_data.columns[0:]

numeric_features = []
categorical_features = []

for dtype, feature in zip(train_data.dtypes[0:], train_data.columns[0:]):
    if dtype == object:
        #print(column)
        #print(train_data[column].describe())
        categorical_features.append(feature)
    else:
        numeric_features.append(feature)
categorical_features

['title', 'director', 'actor1', 'actor2', 'country']

In [85]:
del categorical_features[4]

In [86]:
categorical_features

['title', 'director', 'actor1', 'actor2']

In [87]:
np.random.seed(13)

def impact_coding(data, feature, target='genre'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
    
    In this implementation the KFolds use shuffling. If you want reproducibility the cv 
    could be moved to a parameter.
    '''
    n_folds = 20
    n_inner_folds = 10
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature]):
            impact_coded_cv = pd.Series()
            kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
            inner_split = 0
            inner_oof_mean_cv = pd.DataFrame()
            oof_default_inner_mean = data.iloc[infold][target].mean()
            for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
                # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
                oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
                impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                            lambda x: oof_mean[x[feature]]
                                      if x[feature] in oof_mean.index
                                      else oof_default_inner_mean
                            , axis=1))

                # Also populate mapping (this has all group -> mean for all inner CV folds)
                inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
                inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
                inner_split += 1

            # Also populate mapping
            oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
            oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
            split += 1
            
            impact_coded = impact_coded.append(data.iloc[oof].apply(
                            lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                      if x[feature] in inner_oof_mean_cv.index
                                      else oof_default_mean
                            , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

# Apply the encoding to training and test data, and preserve the mapping
impact_coding_map = {}
for f in categorical_features:
    print("Impact coding for {}".format(f))
    train_data["impact_encoded_{}".format(f)], impact_coding_mapping, default_coding = impact_coding(train_data, f)
    impact_coding_map[f] = (impact_coding_mapping, default_coding)
    mapping, default_mean = impact_coding_map[f]
    test_data["impact_encoded_{}".format(f)] = test_data.apply(lambda x: mapping[x[f]]
                                                                         if x[f] in mapping
                                                                         else default_mean
                                                               , axis=1)

Impact coding for title


C:\Users\vijay\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vijay\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Impact coding for director
Impact coding for actor1
Impact coding for actor2


In [88]:
train_data[['genre','year', 'actor1'] + list(train_data.columns[-8:])]

,genre,year,actor1,actor2,genre,rating,country,impact_encoded_title,impact_encoded_director,impact_encoded_actor1,impact_encoded_actor2
0,5,1969,Antoinette Maynard,Jeanette Mills,5,3.9,USA,5.118250,7.433252,5.117991,5.118078
1,2,2008,Amy Lyndon,Wes Ramsey,2,3.1,USA,2.417513,8.473004,2.417753,2.416980
2,12,2016,Phil Stevens,David Chopping,12,5.8,USA,11.417513,5.971305,11.417444,11.416797
3,6,2002,Matt Dillon,James Caan,6,6.0,USA,6.018105,5.971264,5.808333,7.511905
4,0,2012,Lorin Alond Ly,Thai Ngo,0,9.1,USA,0.617564,4.000000,0.617254,0.616854
5,8,2015,Keke Palmer,Cory Hardrict,8,6.8,USA,8.000000,5.933333,7.817753,6.500000
6,5,2011,Carlen Altman,Bob Byington,5,6.7,USA,5.117029,5.510000,5.117167,5.117579
7,5,2012,Jordan Black,Alice Johnson Boher,5,7.3,USA,5.116995,5.971304,5.117164,5.116716
8,5,2016,Max Azulay,Matt Porter,5,8.1,USA,5.116499,5.971323,5.117991,5.116671
9,5,2013,Miles Teller,Justin Chon,5,5.9,USA,5.117714,4.000000,4.010000,5.117513


In [89]:
X_train = train_data[['impact_encoded_actor1', 'impact_encoded_actor2', 'impact_encoded_title','year','impact_encoded_director','rating']]
y_train = train_data[['genre']]
X_test= test_data[['impact_encoded_actor1', 'impact_encoded_actor2', 'impact_encoded_title','year','impact_encoded_director','rating']]
y_test = test_data[['genre']]

In [91]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=500, max_depth=20, random_state=1, )
random_forest.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_predict = random_forest.predict(X_test)
print(accuracy_score(y_test, y_predict)*100)

C:\Users\vijay\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


35.8982511924
